In [1]:
import os
import difflib
import collections
from datetime import datetime
import openpyxl

In [49]:
# enter basic informations like checkpoint_path and configs_path

ckpt_path = "/root/ext_data/studiogan_ckpt/CIFAR10/checkpoints"
cfg_path = "/root/code/joong/PyTorch-StudioGAN/src/configs/CIFAR10"
dataset_path = "/root/ext_data/data/CIFAR10"
dataset = "CIFAR10_efficient"
num_eval = 1

ckpt_list = os.listdir(ckpt_path)
cfg_list = os.listdir(cfg_path)

# load Excel File and read informations
wb = openpyxl.load_workbook("Taxonomy_experiments_offline.xlsx")
ws = wb['Efficient-CIFAR10-100%']

runs = collections.defaultdict(lambda : collections.defaultdict(str))

all_columns = ws.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        assert col[6].value in ckpt_list, str(col[6].value) + " Not in ckpt_list"
        runs[col[1].value]["run_1"] = col[6].value
        if col[7].value != None:
            assert col[7].value in ckpt_list, str(col[7].value) + " Not in ckpt_list"
            runs[col[1].value]["run_2"] = col[7].value
        if col[8].value != None:
            assert col[8].value in ckpt_list, str(col[8].value) + " Not in ckpt_list"
            runs[col[1].value]["run_3"] = col[8].value
            
        if col[1].value == "ACGAN":
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value+"-Mod", cfg_list, n=1, cutoff=0)[0]
        else:
            runs[col[1].value]["cfg"] = difflib.get_close_matches(col[1].value, cfg_list, n=1, cutoff=0)[0]
        print(f"{col[1].value} matched with {runs[col[1].value]['cfg']}")

SNGAN-DiffAug matched with SNGAN-DiffAug.yaml
SNGAN-ADA matched with SNGAN-ADA.yaml
SNGAN-APA matched with SNGAN-APA.yaml
SNGAN-LeCam matched with SNGAN-LeCam.yaml
SNGAN-DiffAug-LeCam matched with SNGAN-DiffAug-LeCam.yaml
BigGAN-DiffAug matched with BigGAN-DiffAug.yaml
BigGAN-ADA matched with BigGAN-ADA.yaml
BigGAN-APA matched with BigGAN-APA.yaml
BigGAN-LeCam matched with BigGAN-LeCam.yaml
BigGAN-DiffAug-LeCam matched with BigGAN-DiffAug-LeCam.yaml
StyleGAN2-DiffAug matched with StyleGAN2-DiffAug.yaml
StyleGAN2-ADA matched with StyleGAN2-ADA.yaml
StyleGAN2-APA matched with StyleGAN2-APA.yaml
StyleGAN2-LeCam matched with StyleGAN2-LeCam.yaml
StyleGAN2-DiffAug-LeCam matched with StyleGAN2-DiffAug-LeCam.yaml


In [50]:
# Check if evey excel column is matched with proper configurations and modify config if needed.
# runs["TACGAN"]["cfg"] = "ACGAN-Mod-TAC.yaml"
# runs["CRGAN"]["cfg"] = "BigGAN-CR.yaml"
# runs["ICRGAN"]["cfg"] = "BigGAN-ICR.yaml"
# runs["ADCGAN"]["cfg"] = "ACGAN-Mod-Big-ADC.yaml"
# runs["BigGAN-LeCam"]["cfg"] = "BigGAN-256Res-LeCam.yaml"
# runs["StyleGAN2-LeCam"]["cfg"] = "StyleGAN2-256Res-LeCam.yaml"

print("After fixing...")
for key, value in runs.items():
    print(f"{key} matched with {value['cfg']}")

After fixing...
SNGAN-DiffAug matched with SNGAN-DiffAug.yaml
SNGAN-ADA matched with SNGAN-ADA.yaml
SNGAN-APA matched with SNGAN-APA.yaml
SNGAN-LeCam matched with SNGAN-LeCam.yaml
SNGAN-DiffAug-LeCam matched with SNGAN-DiffAug-LeCam.yaml
BigGAN-DiffAug matched with BigGAN-DiffAug.yaml
BigGAN-ADA matched with BigGAN-ADA.yaml
BigGAN-APA matched with BigGAN-APA.yaml
BigGAN-LeCam matched with BigGAN-LeCam.yaml
BigGAN-DiffAug-LeCam matched with BigGAN-DiffAug-LeCam.yaml
StyleGAN2-DiffAug matched with StyleGAN2-DiffAug.yaml
StyleGAN2-ADA matched with StyleGAN2-ADA.yaml
StyleGAN2-APA matched with StyleGAN2-APA.yaml
StyleGAN2-LeCam matched with StyleGAN2-LeCam.yaml
StyleGAN2-DiffAug-LeCam matched with StyleGAN2-DiffAug-LeCam.yaml


In [51]:
# write script file
i = 1
with open("eval_"+dataset+".sh", "w") as f:
    f.write("wandb offline\n")
    train_ = " -ref train "
    metrics_full = " -metrics is fid prdc "
    metrics_fide = " -metrics is fid "
    eval_backbone_incep = " --eval_backbone InceptionV3_tf "
    eval_backbone_swav = " --eval_backbone SwAV_torch "
    eval_backbone_swin = " --eval_backbone Swin-T_torch "
    
    if dataset == "ImageNet":
        prefix_ = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -best --seed 1234 -mpc -DDP" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref valid "
    elif dataset in ["Baby_ImageNet", "Papa_ImageNet", "Grandpa_ImageNet", "AFHQv2"]:
        prefix_S = "CUDA_VISIBLE_DEVICES=0,1,2,3 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 256 -std_step 256 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_B = "CUDA_VISIBLE_DEVICES=0,1,2,3 python src/main.py --pre_resizer lanczos --post_resizer tailored -sync_bn -std_stat -std_max 1024 -std_step 1024 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0,1 python src/main.py --pre_resizer lanczos --post_resizer tailored -best --seed 1234 -mpc -DDP" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref valid "
    elif dataset == "CIFAR10":
        prefix_ = "CUDA_VISIBLE_DEVICES=0 python src/main.py --pre_resizer wo_resize --post_resizer tailored -std_stat -std_max 128 -std_step 128 -best --seed 1234" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        prefix_style = "CUDA_VISIBLE_DEVICES=0 python src/main.py --pre_resizer wo_resize --post_resizer tailored -best --seed 1234 -mpc" + " -data " + dataset_path + " -save ../studiogan/" + " --num_eval " + str(num_eval)
        eval_ = " -ref test "
    
    for (key, value) in runs.items():
        others1 = " -ckpt " + os.path.join(ckpt_path, value['run_1']) + " -cfg " + os.path.join(cfg_path, value['cfg'])
        if value['run_2'] != '':
            others2 = " -ckpt " + os.path.join(ckpt_path, value['run_2']) + " -cfg " + os.path.join(cfg_path, value['cfg'])
        else:
            others2 = None
        if value['run_3'] != '':
            others3 = " -ckpt " + os.path.join(ckpt_path, value['run_3']) + " -cfg " + os.path.join(cfg_path, value['cfg'])
        else:
            others3 = None
        
        if "Style" in key:
            prefix = prefix_style
        elif dataset in ["Baby_ImageNet","Papa_ImageNet", "Grandpa_ImageNet", "AFHQv2"] and ("SAGAN" in key or "SNGAN" in key):
            prefix = prefix_S
        elif dataset in ["Baby_ImageNet","Papa_ImageNet", "Grandpa_ImageNet", "AFHQv2"] and ("BigGAN" in key or "ContraGAN" in key or "ReACGAN" in key):
            prefix = prefix_B
        else:
            prefix = prefix_
        
        for other in [others1, others2, others3]:
            if other != None:
                f.write(prefix + other + eval_backbone_incep + metrics_fide + eval_ + "\n")
                f.write(f"echo =================={i}/{60} completed =============\n")
                i+=1 
                f.write(prefix + other + eval_backbone_swav + metrics_full + train_+ "\n")
                f.write(f"echo =================={i}/{60} completed =============\n")
                i+=1 
                f.write(prefix + other + eval_backbone_swav + metrics_fide + eval_+ "\n")
                f.write(f"echo =================={i}/{60} completed =============\n")
                i+=1 
                f.write(prefix + other + eval_backbone_swin + metrics_full + train_+ "\n")
                f.write(f"echo =================={i}/{60} completed =============\n")
                i+=1 
                f.write(prefix + other + eval_backbone_swin + metrics_fide + eval_+ "\n")
                f.write(f"echo =================={i}/{60} completed =============\n")
                i+=1 

In [ ]:
#### RUN 
#### bash eval_clean_CIFAR10.sh
#### bash eval....sh

In [ ]:
import pickle
with open("/root/code/joong/PyTorch-StudioGAN/eval_pickles/DCGAN-train-2022_01_11_20_32_07---clean.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
print(data)
print(float(data["IS"]))

In [ ]:
def fill_excel(col, start_row, data):
    col[start_row].value = float(data["IS"])
    col[start_row+4].value = data["FID"]
    col[start_row+8].value = data["Improved_Precision"]
    col[start_row+12].value = data["Improved_Recall"]
    col[start_row+16].value = data["Density"]
    col[start_row+20].value = data["Coverage"]
    
def modify_name(col, resize_fn):
    if col[6].value == "CIFAR10-BigGAN-Deep-train-2022_02_02_21_56_10":
        name1 = "BigGAN-Deep-StudioGAN-train-2022_02_02_21_56_10"+ "---"+resize_fn+".pickle"
        name2 = "BigGAN-Deep-StudioGAN-train-2022_02_02_21_56_49"+ "---"+resize_fn+".pickle"
        name3 = "BigGAN-Deep-StudioGAN-train-2022_02_02_22_01_49"+ "---"+resize_fn+".pickle"
    elif col[6].value[8:].startswith("BigGAN-Mod"):
        name1 = col[6].value[8:][0:6]+col[6].value[8:][10:]+ "---"+resize_fn+".pickle"
        name2 = col[7].value[8:][0:6]+col[7].value[8:][10:]+ "---"+resize_fn+".pickle"
        name3 = col[8].value[8:][0:6]+col[8].value[8:][10:]+ "---"+resize_fn+".pickle"
    else:
        name1 = col[6].value[8:] + "---"+resize_fn+".pickle"
        name2 = col[7].value[8:] + "---"+resize_fn+".pickle"
        name3 = col[8].value[8:] + "---"+resize_fn+".pickle"
    return name1, name2, name3


In [ ]:
# load Excel File and read informations
wb_clean = openpyxl.load_workbook("CIFAR10_clean.xlsx")
ws_clean = wb_clean['Sheet1']
pickles_path = "/root/code/joong/PyTorch-StudioGAN/eval_pickles/"
pickles_list = os.listdir(pickles_path)

all_columns = ws_clean.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        name1, name2, name3 = modify_name(col, "clean")
        assert name1 in pickles_list, name1 + " Pickle Does not exist"
        assert name2 in pickles_list, name2 + " Pickle Does not exist"
        assert name3 in pickles_list, name3 + " Pickle Does not exist"

        with open(os.path.join(pickles_path, name1), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 10, data)
        with open(os.path.join(pickles_path, name2), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 11, data)
        with open(os.path.join(pickles_path, name3), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 12, data)
wb_clean.save("CIFAR10_clean.xlsx")

In [ ]:
# load Excel File and read informations
wb_legacy = openpyxl.load_workbook("CIFAR10_legacy.xlsx")
ws_legacy = wb_legacy['Sheet1']
pickles_path = "/root/code/joong/PyTorch-StudioGAN/eval_pickles/"
pickles_list = os.listdir(pickles_path)

all_columns = ws_legacy.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        name1, name2, name3 = modify_name(col, "legacy")
        assert name1 in pickles_list, name1 + " Pickle Does not exist"
        assert name2 in pickles_list, name2 + " Pickle Does not exist"
        assert name3 in pickles_list, name3 + " Pickle Does not exist"

        with open(os.path.join(pickles_path, name1), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 10, data)
        with open(os.path.join(pickles_path, name2), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 11, data)
        with open(os.path.join(pickles_path, name3), "rb") as f:
            data = pickle.load(f)
            fill_excel(col, 12, data)
wb_legacy.save("CIFAR10_legacy.xlsx")

In [ ]:
import pickle
import os
pickles_path = "/root/code/joong/PyTorch-StudioGAN/eval_pickles/"
pickles_list = sorted(os.listdir(pickles_path))
for item in pickles_list:
    if item.endswith("le"):
        with open(pickles_path+str(item), "rb") as f:
            data = pickle.load(f)
            if data == {}:
                print(item[:-7])
            # if "Improved_Precision" not in data.keys():
            #     print(item)


In [12]:
pickles_path = "/root/code/joong/PyTorch-StudioGAN/eval_pickles/"
pickles_list = sorted(os.listdir(pickles_path))

for item in pickles_list:
    if item.endswith(".pickle"):
        os.rename(pickles_path+item, pickles_path+item[:-7]+"-train.pickle")

In [ ]:
# enter basic informations like checkpoint_path and configs_path,
# load Excel File and read informations
import openpyxl
wb = openpyxl.load_workbook("Taxonomy_experiments_offline.xlsx")
ws = wb['Efficient-AFHQ-V2']
dataset_name = 'AFHQv2-'
pickles_path = "/root/code/joong/PyTorch-StudioGAN/eval_pickles/"
pickles_list = sorted(os.listdir(pickles_path))

all_columns = ws.columns
for idx,col in enumerate(all_columns):
    if idx >=2 and col[1].value != None:
        run_name1 = col[6].value[len(dataset_name):]
        run_name2 = col[7].value[len(dataset_name):] if col[7].value is not None else None
        run_name3 = col[8].value[len(dataset_name):] if col[8].value is not None else None
        for run_name  in [run_name1, run_name2, run_name3]:
            if run_name is not None:
                if run_name + ".pickle" in pickles_list:
                    os.rename(pickles_path + run_name + ".pickle", pickles_path + dataset_name + run_name + ".pickle")
                elif dataset_name + run_name + ".pickle" in pickles_list:
                    pass
                else:
                    print("error", run_name + ".pickle" , "not in pickles_list")
        
            
